<a href="https://colab.research.google.com/github/isegura/OCW-UC3M-NLPDeep-2023/blob/main/tema4_4__matriz_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# 4.4 ¿Cómo crear una matriz de word embeddings a partir de un modelo de word embeddings?



En los temas 2 y 3, aprendimos a entrenar modelos de deep learning, en particular CNN y BiLSTM, para la tarea de clasificación de textos. En dichos modelos, los tokens eran reprsentados por vectores que se inicializaban de forma aleatoria en la capa Embedding.

En este ejercicio, aprenderemos a crear una matriz de embeddings a partir de un modelo de word embeddings. La matriz tendrá tantas filas como número de tokens distintos en la colección de textos utilizada para entrenar un modelo de deep learning. Cada fila representa el word embedding de un determinado token, que será de un modelo pre-entrenado de word embedding.
El número de columnas será igual a la dimensión de los embeddings en dicho modelo pre-entrenado.


De esta forma, cuando vayamos a inicializar los vectores de los tokens en nuestro modelo de deep learning, en lugar de hacerlo de forma aleatoria, usaremos dicha matriz.

El dataset a utilizar es la colección de textos proporcionada por los organizadores de la competición EXIST 2021, cuyo principal objetivo era detectar mensajes sexistas en las redes sociales.

Este dataset puede solicitarse para usos en investigación en el siguiente http://nlp.uned.es/exist2021/.



**NOTA PARA PODER EJECUTAR ESTE NOTEBOOK**:

1) Para poder ejercutar correctamente este notebook, deberás abrirlo en tu Google Drive (por ejemplo, en la carpeta 'Colab Notebooks').

2) Además, debes guardar el dataset en tu Google Drive, dentro de carpeta 'Colab Notebooks/data/exist2021/'.


## Cargar el dataset
Lo primero que debes hacer es cargar el dataset. Como hemos comentado antes, recuerda que para que puedas ejecutar este notebook, deberás haber descargado los ficheros del dataset y haberlos almacenados en tu carpeta 'Colab Notebooks/data/exist2021/' dentro de tu Gooogle Drive.

In [28]:
from google.colab import drive
# monta tu unidad de google drive
drive.mount('/content/drive')

# cambiamos el directorio de trabajo a la carpeta donde está almacenado el dataset
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/data/exist2021/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


En este ejercicio, únicamente vamos a cargar el dataset de entrenamiento:

In [29]:
import pandas as pd

df_train = pd.read_csv("train.tsv",  sep='\t')
print('tamaño training:', df_train.shape)

# mostramos los primeros registros del training:
df_train.head()

tamaño training: (6977, 7)


,test_case,id,source,language,text,task1,task2
0,EXIST2021,1,twitter,en,"She calls herself ""anti-feminazi"" how about sh...",sexist,ideological-inequality
1,EXIST2021,2,twitter,en,"Now, back to these women, the brave and the be...",non-sexist,non-sexist
2,EXIST2021,3,twitter,en,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...",sexist,objectification
3,EXIST2021,4,twitter,en,@AurelieGuiboud Incredible! Beautiful!But I l...,non-sexist,non-sexist
4,EXIST2021,5,twitter,en,i find it extremely hard to believe that kelly...,non-sexist,non-sexist


Es un dataset multilingüe. Nos vamos a quedar únicamente con los textos en inglés, porque el modelo pre-entrenado de word embeddings que vamos a utilizar en este ejercicio es un modelo que sólo contiene palabras en inglés:

In [30]:
df_train = df_train.loc[df_train['language'] == 'en']
df_train.shape

(3436, 7)

Como hemos dicho antes la matriz tendrá una dimensión de |V| x d, donde V es el vocabulario del conjunto de entrenamiento (es decir, el conjunto de tokens distintos en la colección de textos) y d la dimensión de los word embeddings del modelos pre-entrenado de word embeddings que cargaremos más tarde.

Para obtener el vocabulario V, vamos a tokenizar la colección de textos. Para ello vamos a utilizar el tokenizador de Keras:


In [31]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer = Tokenizer(oov_token = True)
X_train = list(df_train['text'])

tokenizer.fit_on_texts(X_train)
tokenizer.word_index['<PAD>'] = 0

NUM_WORDS=len(tokenizer.word_index)
print("Tamaño del vocabulario ={}".format(NUM_WORDS))



Tamaño del vocabulario =16645


El siguiente paso será cargar el modelo pre-entrenado de word embeddings. En concreto, vamos a utilizar el modelo **glove-wiki-gigaword-50**, que es relativamente pequeño (y nos permitirá ejecutar este notebook en un tiempo corto). Para descargarlo y cargarlo usaremos la librería Gensim.

Puedes probar con otros modelos de word embeddings (https://github.com/RaRe-Technologies/gensim-data).

In [32]:
import gensim.downloader as api

model_we = api.load("glove-wiki-gigaword-50")


La dimensión de los vectores de ese modelo es 50, pero de todas formas, vamos a obtenerlo de forma automática. Para ello recuperamos el vector de alguna palabra común en inglés, por ejemplo, house (que seguro está en el modelo). Guardamos dicho valor en la variable EMBED_SIZE.

In [33]:
wv = model_we['house']
print(wv.shape)
EMBED_SIZE=wv.shape[0]
print(EMBED_SIZE)


(50,)
50


Una vez cargado el modelo, continuamos creando una matriz de tamaño |V| x d, donde |V| ya hemos visto que es igual a 16645, y d es 50 (la dimensión de los vectores en el modelo glove-wiki-gigaword-50). La matriz la inicializamos a 0.


In [34]:
import numpy as np
embedding_matrix = np.zeros((NUM_WORDS, EMBED_SIZE))
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

El siguiente paso ya sí es modificar la matriz con los vectores del modelo de word embeddings.
Cada fila de la matriz será el word embeddings de un determinado token en el vocabulario V.

Para acceder a las palabras del vocabulario, usamos el tokenizador que acabamos de entrenar con la colección de textos.
```
tokenizer.word_index.items()
```
Los tokens están ordenados por frecuencia (excepto el primer token que es pad) y son representados por un índice (que será también el índice de su fila en la matriz).

Debemos recorrer los tokens en dicho vocabulario, y vamos a mirar si en el modelo pre-entrenado de word embeddings existe un vector para dicho token. Si existe, reemplazamos la fila i por dicho vector, donde i es además el índice del token en el vocabulario.
Si dicho token no existe en el modelo pre-entrenado, no modificamos la matriz (la fila para esa palabra seguirá siendo un vector de 0's).  


In [35]:
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_we[word]
        # word embedding para la palabra con índice i
        embedding_matrix[i] = embedding_vector
    except:
        #if word does not exist, we do not udpate the matrix
        pass

print('matriz creada')

embedding_matrix

matriz creada


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.41800001,  0.24968   , -0.41242   , ..., -0.18411   ,
        -0.11514   , -0.78580999],
       ...,
       [-0.18762   , -0.55958003, -0.12295   , ...,  0.30331999,
        -0.61690998,  0.4109    ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

Mostramos las 10 primeras palabras del vocabulario y sus vectores. La primera fila es la fila del token con índice 0, que es el token PAD.

In [36]:
print(embedding_matrix.shape)
for word, i in tokenizer.word_index.items():
    print(word,  embedding_matrix[i])
    if i==10:
        break


(16645, 50)
True [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
the [ 4.18000013e-01  2.49679998e-01 -4.12420005e-01  1.21699996e-01
  3.45270008e-01 -4.44569997e-02 -4.96879995e-01 -1.78619996e-01
 -6.60229998e-04 -6.56599998e-01  2.78430015e-01 -1.47670001e-01
 -5.56770027e-01  1.46579996e-01 -9.50950012e-03  1.16579998e-02
  1.02040000e-01 -1.27920002e-01 -8.44299972e-01 -1.21809997e-01
 -1.68009996e-02 -3.32789987e-01 -1.55200005e-01 -2.31309995e-01
 -1.91809997e-01 -1.88230002e+00 -7.67459989e-01  9.90509987e-02
 -4.21249986e-01 -1.95260003e-01  4.00710011e+00 -1.85939997e-01
 -5.22870004e-01 -3.16810012e-01  5.92130003e-04  7.44489999e-03
  1.77780002e-01 -1.58969998e-01  1.20409997e-02 -5.42230010e-02
 -2.98709989e-01 -1.57490000e-01 -3.47579986e-01 -4.56370004e-02
 -4.42510009e-01  1.87849998e-01  2.78489990e-03 -1.84110001e-01
 -1.15139998e-01 -7.85809994e-01]
to [ 0.6804

Ahora ya podríamos utilizar la matriz para inicializar los vectores en un modelo de deep learning.
Para ello simplemente, en la capa de Embeddings del modelo deberemos pasar dicha matriz al argumento **weights**.
                     

```
# Capa embeddings
model.add(Embedding(NUM_WORDS, EMBED_SIZE,
                        input_length=MAX_LEN,
                        weights=[embedding_matrix]))
```

Consulta las prácticas de este curso OCW para encontrar ejemplos concretos sobre esta inicialización.